In [1]:
import torch
import pandas as pd
import shapely
import numpy as np
import sys
import torch
import argparse
import random
import math
import os

from sklearn.preprocessing import OneHotEncoder
from scipy.stats import stats
from torch.utils.data import DataLoader
from torch.optim import lr_scheduler

In [2]:
# read file
#data = pd.read_pickle('../data/thread_0_0_999.pkl')
data = pd.read_pickle('../sample.tmp')

In [3]:
df = data.copy()

## test the usability of rnn in this problem
- make sure the target is predictable

to do this, have to extract some feature first:
- Teile Nr.
- seq in job

In [5]:
# num_of_corr is the teile Nr
data['num_of_corr'] = data['List of Coordinates'].map(lambda x: len(x))
# seq_in_job
data['seq_in_job'] = data['Jobid'].map(lambda x: x.split('_')[-1])
#data.sort_values(['seq_in_job', 'num_of_corr'])

## prepare the data
what kinds of data do i need
- vector representation of the 'Teile' 
- vector representation of the rotation

In [4]:
df['num_of_corr'] = df['List of Coordinates'].map(lambda x: len(x))

In [5]:
# get onehot data for 'Teile-Nr'
df = df.reset_index().iloc[:,1:]
# get onehot data
ohe = OneHotEncoder()
tmp = ohe.fit_transform(df['num_of_corr'].values.reshape(-1, 1)).toarray()
tmp = pd.DataFrame(tmp)
tmp.columns = ['s1', 's2', 's3', 's4', 's5', 's6']
df = pd.concat([df, tmp], axis = 1)

/anaconda3/envs/data_mining/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [6]:
# get onehot data for rotation
df['Rot'][df['Rot'] == '5.00'] = '0.00'
# get onehot data
tmp = ohe.fit_transform(df['Rot'].values.reshape(-1, 1)).toarray()
tmp = pd.DataFrame(tmp)
tmp.columns = ['r1', 'r2']
df = pd.concat([df, tmp], axis = 1)

/anaconda3/envs/data_mining/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [7]:
# change type of clumns for y
df['Metric2'] = pd.to_numeric(df['Metric2'])
df['Metric3'] = pd.to_numeric(df['Metric3'])
df['Metric4'] = pd.to_numeric(df['Metric4'])

In [38]:
# test
df[1200:1300]

,Jobid,Rot,List of Coordinates,Shapely Polygon,Metric1,Metric2,Metric3,Metric4,Area,Length,Left down,num_of_corr,s1,s2,s3,s4,s5,s6,r1,r2
1200,92600-2514_5,90.00,"[(6.0, 38.0), (7.1, 40.0), (97.5, 40.0), (98.7...","POLYGON ((6 38, 7.1 40, 97.5 40, 98.7 38, 98.7...",15.00,1.0,967.0,2.0,23178.165,1448.933262,"(41.0, 967.0)",17,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
1201,92600-2514_5,0.00,"[(6.0, 38.0), (7.1, 40.0), (97.5, 40.0), (98.7...","POLYGON ((6 38, 7.1 40, 97.5 40, 98.7 38, 98.7...",16.00,693.0,1.0,5.0,23178.165,1448.933262,"(1381.3, 1.0)",17,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1202,92600-2514_5,90.00,"[(0.0, 9.0), (9.0, 0.0), (115.8, 0.0), (124.8,...","POLYGON ((0 9, 9 0, 115.8 0, 124.8 9, 124.8 45...",17.00,1.0,1010.0,4.0,4439.800,615.763412,"(55.5, 1010.0)",18,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
1203,92600-2514_5,90.00,"[(117.4, 156.1), (117.4, 108.0), (78.2, 108.0)...","POLYGON ((117.4 156.1, 117.4 108, 78.2 108, 78...",18.00,1.0,1067.0,4.0,16412.140,1145.674013,"(157.1, 1067.0)",26,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
1204,92600-2514_5,90.00,"[(0.0, 9.0), (9.0, 0.0), (115.8, 0.0), (124.8,...","POLYGON ((0 9, 9 0, 115.8 0, 124.8 9, 124.8 45...",19.00,1.0,1226.0,4.0,4439.800,615.763412,"(55.5, 1226.0)",18,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
1205,92600-2514_5,0.00,"[(117.4, 156.1), (117.4, 108.0), (78.2, 108.0)...","POLYGON ((117.4 156.1, 117.4 108, 78.2 108, 78...",20.00,1.0,1283.0,4.0,16412.140,1145.674013,"(118.4, 1283.0)",26,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1206,92600-2514_5,0.00,"[(6.0, 38.0), (7.1, 40.0), (97.5, 40.0), (98.7...","POLYGON ((6 38, 7.1 40, 97.5 40, 98.7 38, 98.7...",21.00,693.0,692.0,5.0,23178.165,1448.933262,"(1381.3, 692.0)",17,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1207,92600-2514_5,90.00,"[(117.4, 156.1), (117.4, 108.0), (78.2, 108.0)...","POLYGON ((117.4 156.1, 117.4 108, 78.2 108, 78...",22.00,737.0,1.0,4.0,16412.140,1145.674013,"(893.1, 1.0)",26,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
1208,92600-2514_5,90.00,"[(28.5, 0.0), (60.1, 0.0), (84.4, 7.9), (88.3,...","POLYGON ((28.5 0, 60.1 0, 84.40000000000001 7....",23.00,1.0,1403.0,4.0,1103.760,192.718763,"(21.0, 1403.0)",13,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1209,92600-2514_5,90.00,"[(1.0, 123.3), (0.0, 122.3), (0.0, 93.3), (1.0...","POLYGON ((1 123.3, 0 122.3, 0 93.3, 1 92.3, 21...",24.00,705.0,1325.0,4.0,7228.382,901.580393,"(828.3, 1325.0)",54,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0


In [11]:
# test
tmp = df[['Jobid', 'Metric4']].groupby('Jobid', as_index = False).mean()

In [12]:
# test
tmp.describe()

,Metric4
count,1000000.000000
mean,3.393246
std,1.244949
min,2.000000
25%,2.000000
50%,4.000000
75%,4.000000
max,5.000000


(15402, 50, 8)

In [22]:
# combine the sequence in one task and transform to numpy
df['Jobid'] = df['Jobid'].map(lambda x: x.split('_')[0]+'_'+x.split('_')[1])

x = []
y = []
for name, group in df.groupby('Jobid'):
    if len(group) == 50:
        x.append(group.iloc[:, 12:].values)
        y.append(group.iloc[:, 5:8].values)

x = np.stack(x, axis = 0)
y = np.stack(y, axis = 0)

In [24]:
x = np.float32(x)
y = np.float32(y)

In [25]:
# distribute the data
train_x = x[:int(len(x)*0.9)]
test_x = x[int(len(x)*0.9):]
train_y = y[:int(len(x)*0.9)]
test_y = y[int(len(x)*0.9):]

In [26]:
# save to the local system
np.save('../data/rnn_train_x', train_x)
np.save('../data/rnn_train_y', train_y)
np.save('../data/rnn_test_x', test_x)
np.save('../data/rnn_test_y', test_y)

## bild the rnn model

In [35]:
# bild the model, loss and data class
class TS_rnn(torch.nn.Module):
    """
    scores for each piece
    input:
        tensor size of (batch_size, seq_len, num_dim)
    output:
        tensor size of (batch_size, seq_len)
    """
    def __init__(self):
        super(TS_rnn, self).__init__()
        #change the structure of the network
        num_inp = 8
        num_hidden = 64
        self.rnn = torch.nn.LSTM(input_size = num_inp, hidden_size = num_hidden, num_layers = 2)
        self.mlp = torch.nn.Sequential(
                torch.nn.Linear(num_hidden, 16),
                torch.nn.Dropout(),
                torch.nn.ReLU(),
                torch.nn.Linear(16, 1)
                )

    def forward(self, inp):
        # input of the rnn (seq_len, batch, input_size)
        data_in = torch.transpose(inp, 0, 1)
        # run rnn, it has two output
        out_rnn, _ = self.rnn(data_in)
        out_rnn = torch.transpose(out_rnn, 0, 1) # (batch_size, seq_len, num_dim)
        # rnn the mlp
        batch_size, seq_len, num_dim = out_rnn.shape
        out = []
        for i in range(seq_len):
            tmp = self.mlp(out_rnn[:, i,:])
            out.append(tmp)
        # now out is list of (batch_size, 1), combine the items in the list to get the output with size (batch_size, seq_len)
        out = torch.cat(out, 1)
        return out.squeeze()

class TS_rnn2(torch.nn.Module):
    """
    scores only for the whole task
    input:
        tensor size of (batch_size, seq_len, num_dim)
    output:
        tensor size of (batch_size)
    """
    def __init__(self):
        super(TS_rnn2, self).__init__()
        #change the structure of the network
        num_inp = 8
        num_hidden = 64
        self.rnn = torch.nn.LSTM(input_size = num_inp, hidden_size = num_hidden, num_layers = 2)
        self.mlp = torch.nn.Sequential(
                torch.nn.Linear(num_hidden, 64),
                torch.nn.Dropout(),
                torch.nn.ReLU(),
                torch.nn.Linear(64, 1)
                )

    def forward(self, inp):
        # input of the rnn (seq_len, batch, input_size)
        data_in = torch.transpose(inp, 0, 1)
        # run rnn, it has two output
        out_rnn, _ = self.rnn(data_in)
        out_rnn = torch.transpose(out_rnn, 0, 1) # (batch_size, seq_len, num_dim)
        # only use the last output
        out_rnn = out_rnn[:, -1, :].squeeze()
        # rnn the mlp
        out = self.mlp(out_rnn)
        return out.squeeze()
    
class PDLoss(torch.nn.Module):
    def __init__(self, p = 2):
        super(PDLoss, self).__init__()
        self.pd = torch.nn.PairwiseDistance(p)

    def forward(self, o, t):
        # out: (batch_size, 1)
        out = self.pd(o, t)
        return out.mean()

class Data:
    """
    data class for TS_rnn
    """
    def __init__(self, x, y):
        self.data = {}
        self.data['train_x'] = self.add_file(x)
        self.data['train_y'] = self.add_file(y)[:, :, -1] # use the first metric tempately
        assert(len(self.data['train_x']) == len(self.data['train_y']))
        self.len = len(self.data['train_x'])

    def add_file(self, path):
        return torch.from_numpy(np.load(path))

    def add_scores(self, path):
        return torch.FloatTensor([float(li.rstrip('\n')) for li in open(path)])

    def __len__(self):
        return self.len

    def __getitem__(self, index):
        return (self.data['train_x'][index],
                self.data['train_y'][index])

class Data2:
    """
    data class for TS_rnn2
    """
    def __init__(self, x, y):
        self.data = {}
        self.data['train_x'] = self.add_file(x)
        self.data['train_y'] = self.add_file(y)[:, :, -1] # use the first metric tempately
        self.data['train_y'] = torch.mean(self.data['train_y'], 1)
        assert(len(self.data['train_x']) == len(self.data['train_y']))
        self.len = len(self.data['train_x'])

    def add_file(self, path):
        return torch.from_numpy(np.load(path))

    def add_scores(self, path):
        return torch.FloatTensor([float(li.rstrip('\n')) for li in open(path)])

    def __len__(self):
        return self.len

    def __getitem__(self, index):
        return (self.data['train_x'][index],
                self.data['train_y'][index])

In [33]:
# write the test function
def test_model(dl_test, model, loss):
    model.eval()
    test_loss = 0
    counter = 0
    for batch_idx, dat in enumerate(dl_test):
        counter += 1
        # codes to be changed
        inp, target = dat
        out = model(inp)
        lo = loss(out, target.squeeze())
        test_loss += lo.data
    return test_loss/counter

In [37]:
# rnn with 2 hidden layer: 8, 16; mlp with size 16X16X1; L1Loss; epoch 26 test error: 40 (few imporve after 20 epochs?)
# rnn with 2 hidden layer: 8, 16; mlp with size 16X64X1; L1Loss;
# run the model
train_x = '../data/rnn_train_x.npy'
train_y = '../data/rnn_train_y.npy'
test_x = '../data/rnn_test_x.npy'
test_y = '../data/rnn_test_y.npy'
num_epochs = 100
batch_size = 30
verbose = True
# set models and loss
model = TS_rnn()
#loss = PDLoss()
loss = torch.nn.L1Loss()
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3)
# set the scheduler
lamb1 = lambda x: .1**(x//30)
scheduler = lr_scheduler.LambdaLR(optimizer, lr_lambda = lamb1)
# loda data
train = Data(train_x, train_y)
test = Data(test_x, test_y)
dl_train = DataLoader(train, batch_size = batch_size, shuffle = True)
dl_test = DataLoader(test, batch_size = batch_size, shuffle = True)
# train the model
for epoch in range(num_epochs):
    scheduler.step()
    model.train()
    train_loss = 0
    counter = 0
    for batch_idx, dat in enumerate(dl_train):
        counter += 1
        # train the model
        optimizer.zero_grad()
        inp, target = dat
        out = model(inp)
        lo = loss(out, target.squeeze())
        lo.backward()
        optimizer.step()
        train_loss += lo.data
        if verbose:
            if batch_idx % 10 == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch,
                    batch_idx * batch_size,
                    len(train),
                    100.*batch_idx*batch_size/len(train),
                    lo.data
                    ))
    test_lo = test_model(dl_test, model, loss)
    if verbose:
        # train loss
        print('====> Epoch: {} Average train loss: {:.4f}'.format(
            epoch,
            train_loss/counter
            ))
        # test loss
        print('====> Epoch: {} Average test loss: {:.4f}'.format(
            epoch,
            test_lo
            ))

Train Epoch: 0 [0/13861 (0%)]	Loss: 3.192075
Train Epoch: 0 [300/13861 (2%)]	Loss: 2.977371
Train Epoch: 0 [600/13861 (4%)]	Loss: 2.353096
Train Epoch: 0 [900/13861 (6%)]	Loss: 1.586143
Train Epoch: 0 [1200/13861 (9%)]	Loss: 1.585013
Train Epoch: 0 [1500/13861 (11%)]	Loss: 1.466203
Train Epoch: 0 [1800/13861 (13%)]	Loss: 1.455357
Train Epoch: 0 [2100/13861 (15%)]	Loss: 1.445559
Train Epoch: 0 [2400/13861 (17%)]	Loss: 1.472007
Train Epoch: 0 [2700/13861 (19%)]	Loss: 1.432854
Train Epoch: 0 [3000/13861 (22%)]	Loss: 1.420577
Train Epoch: 0 [3300/13861 (24%)]	Loss: 1.393119
Train Epoch: 0 [3600/13861 (26%)]	Loss: 1.373754
Train Epoch: 0 [3900/13861 (28%)]	Loss: 1.386902
Train Epoch: 0 [4200/13861 (30%)]	Loss: 1.410030
Train Epoch: 0 [4500/13861 (32%)]	Loss: 1.406524
Train Epoch: 0 [4800/13861 (35%)]	Loss: 1.377182
Train Epoch: 0 [5100/13861 (37%)]	Loss: 1.329157
Train Epoch: 0 [5400/13861 (39%)]	Loss: 1.424987
Train Epoch: 0 [5700/13861 (41%)]	Loss: 1.362978
Train Epoch: 0 [6000/13861 (43%

Train Epoch: 3 [6600/13861 (48%)]	Loss: 0.629176
Train Epoch: 3 [6900/13861 (50%)]	Loss: 0.648726
Train Epoch: 3 [7200/13861 (52%)]	Loss: 0.651508
Train Epoch: 3 [7500/13861 (54%)]	Loss: 0.656186
Train Epoch: 3 [7800/13861 (56%)]	Loss: 0.665317
Train Epoch: 3 [8100/13861 (58%)]	Loss: 0.644623
Train Epoch: 3 [8400/13861 (61%)]	Loss: 0.644222
Train Epoch: 3 [8700/13861 (63%)]	Loss: 0.627407
Train Epoch: 3 [9000/13861 (65%)]	Loss: 0.655719
Train Epoch: 3 [9300/13861 (67%)]	Loss: 0.625935
Train Epoch: 3 [9600/13861 (69%)]	Loss: 0.615794
Train Epoch: 3 [9900/13861 (71%)]	Loss: 0.644770
Train Epoch: 3 [10200/13861 (74%)]	Loss: 0.647325
Train Epoch: 3 [10500/13861 (76%)]	Loss: 0.637384
Train Epoch: 3 [10800/13861 (78%)]	Loss: 0.616639
Train Epoch: 3 [11100/13861 (80%)]	Loss: 0.642465
Train Epoch: 3 [11400/13861 (82%)]	Loss: 0.590295
Train Epoch: 3 [11700/13861 (84%)]	Loss: 0.645702
Train Epoch: 3 [12000/13861 (87%)]	Loss: 0.617340
Train Epoch: 3 [12300/13861 (89%)]	Loss: 0.620389
Train Epoch:

Train Epoch: 6 [12900/13861 (93%)]	Loss: 0.548438
Train Epoch: 6 [13200/13861 (95%)]	Loss: 0.550439
Train Epoch: 6 [13500/13861 (97%)]	Loss: 0.543149
Train Epoch: 6 [13800/13861 (100%)]	Loss: 0.550667
====> Epoch: 6 Average train loss: 0.5387
====> Epoch: 6 Average test loss: 0.3643
Train Epoch: 7 [0/13861 (0%)]	Loss: 0.546108
Train Epoch: 7 [300/13861 (2%)]	Loss: 0.536227
Train Epoch: 7 [600/13861 (4%)]	Loss: 0.537080
Train Epoch: 7 [900/13861 (6%)]	Loss: 0.534060
Train Epoch: 7 [1200/13861 (9%)]	Loss: 0.568542
Train Epoch: 7 [1500/13861 (11%)]	Loss: 0.551365
Train Epoch: 7 [1800/13861 (13%)]	Loss: 0.548540
Train Epoch: 7 [2100/13861 (15%)]	Loss: 0.555843
Train Epoch: 7 [2400/13861 (17%)]	Loss: 0.521275
Train Epoch: 7 [2700/13861 (19%)]	Loss: 0.553945
Train Epoch: 7 [3000/13861 (22%)]	Loss: 0.542719
Train Epoch: 7 [3300/13861 (24%)]	Loss: 0.533763
Train Epoch: 7 [3600/13861 (26%)]	Loss: 0.542724
Train Epoch: 7 [3900/13861 (28%)]	Loss: 0.556385
Train Epoch: 7 [4200/13861 (30%)]	Loss: 0

Train Epoch: 10 [4800/13861 (35%)]	Loss: 0.525713
Train Epoch: 10 [5100/13861 (37%)]	Loss: 0.521473
Train Epoch: 10 [5400/13861 (39%)]	Loss: 0.554628
Train Epoch: 10 [5700/13861 (41%)]	Loss: 0.526731
Train Epoch: 10 [6000/13861 (43%)]	Loss: 0.552040
Train Epoch: 10 [6300/13861 (45%)]	Loss: 0.532766
Train Epoch: 10 [6600/13861 (48%)]	Loss: 0.527141
Train Epoch: 10 [6900/13861 (50%)]	Loss: 0.532671
Train Epoch: 10 [7200/13861 (52%)]	Loss: 0.552447
Train Epoch: 10 [7500/13861 (54%)]	Loss: 0.533378
Train Epoch: 10 [7800/13861 (56%)]	Loss: 0.538908
Train Epoch: 10 [8100/13861 (58%)]	Loss: 0.546881
Train Epoch: 10 [8400/13861 (61%)]	Loss: 0.524796
Train Epoch: 10 [8700/13861 (63%)]	Loss: 0.535648
Train Epoch: 10 [9000/13861 (65%)]	Loss: 0.552843
Train Epoch: 10 [9300/13861 (67%)]	Loss: 0.539236
Train Epoch: 10 [9600/13861 (69%)]	Loss: 0.539988
Train Epoch: 10 [9900/13861 (71%)]	Loss: 0.524353
Train Epoch: 10 [10200/13861 (74%)]	Loss: 0.508044
Train Epoch: 10 [10500/13861 (76%)]	Loss: 0.54333

Train Epoch: 13 [10200/13861 (74%)]	Loss: 0.544354
Train Epoch: 13 [10500/13861 (76%)]	Loss: 0.523453
Train Epoch: 13 [10800/13861 (78%)]	Loss: 0.546816
Train Epoch: 13 [11100/13861 (80%)]	Loss: 0.552583
Train Epoch: 13 [11400/13861 (82%)]	Loss: 0.528850
Train Epoch: 13 [11700/13861 (84%)]	Loss: 0.542217
Train Epoch: 13 [12000/13861 (87%)]	Loss: 0.518171
Train Epoch: 13 [12300/13861 (89%)]	Loss: 0.525603
Train Epoch: 13 [12600/13861 (91%)]	Loss: 0.557399
Train Epoch: 13 [12900/13861 (93%)]	Loss: 0.546051
Train Epoch: 13 [13200/13861 (95%)]	Loss: 0.548726
Train Epoch: 13 [13500/13861 (97%)]	Loss: 0.528831
Train Epoch: 13 [13800/13861 (100%)]	Loss: 0.574518
====> Epoch: 13 Average train loss: 0.5357
====> Epoch: 13 Average test loss: 0.3926
Train Epoch: 14 [0/13861 (0%)]	Loss: 0.515547
Train Epoch: 14 [300/13861 (2%)]	Loss: 0.525851
Train Epoch: 14 [600/13861 (4%)]	Loss: 0.513448
Train Epoch: 14 [900/13861 (6%)]	Loss: 0.528708
Train Epoch: 14 [1200/13861 (9%)]	Loss: 0.547941
Train Epoch:

...

Train Epoch: 84 [6600/13861 (48%)]	Loss: 0.542275
Train Epoch: 84 [6900/13861 (50%)]	Loss: 0.489050
Train Epoch: 84 [7200/13861 (52%)]	Loss: 0.516081
Train Epoch: 84 [7500/13861 (54%)]	Loss: 0.549721
Train Epoch: 84 [7800/13861 (56%)]	Loss: 0.546077
Train Epoch: 84 [8100/13861 (58%)]	Loss: 0.527642
Train Epoch: 84 [8400/13861 (61%)]	Loss: 0.514215
Train Epoch: 84 [8700/13861 (63%)]	Loss: 0.527347
Train Epoch: 84 [9000/13861 (65%)]	Loss: 0.520968
Train Epoch: 84 [9300/13861 (67%)]	Loss: 0.523303
Train Epoch: 84 [9600/13861 (69%)]	Loss: 0.529314
Train Epoch: 84 [9900/13861 (71%)]	Loss: 0.507110
Train Epoch: 84 [10200/13861 (74%)]	Loss: 0.535547
Train Epoch: 84 [10500/13861 (76%)]	Loss: 0.512035
Train Epoch: 84 [10800/13861 (78%)]	Loss: 0.528168
Train Epoch: 84 [11100/13861 (80%)]	Loss: 0.511071
Train Epoch: 84 [11400/13861 (82%)]	Loss: 0.516636
Train Epoch: 84 [11700/13861 (84%)]	Loss: 0.509305
Train Epoch: 84 [12000/13861 (87%)]	Loss: 0.544475
Train Epoch: 84 [12300/13861 (89%)]	Loss: 0

Train Epoch: 87 [12000/13861 (87%)]	Loss: 0.535821
Train Epoch: 87 [12300/13861 (89%)]	Loss: 0.518225
Train Epoch: 87 [12600/13861 (91%)]	Loss: 0.543594
Train Epoch: 87 [12900/13861 (93%)]	Loss: 0.522554
Train Epoch: 87 [13200/13861 (95%)]	Loss: 0.526134
Train Epoch: 87 [13500/13861 (97%)]	Loss: 0.525370
Train Epoch: 87 [13800/13861 (100%)]	Loss: 0.536183
====> Epoch: 87 Average train loss: 0.5270
====> Epoch: 87 Average test loss: 0.3634
Train Epoch: 88 [0/13861 (0%)]	Loss: 0.563880
Train Epoch: 88 [300/13861 (2%)]	Loss: 0.517501
Train Epoch: 88 [600/13861 (4%)]	Loss: 0.520530
Train Epoch: 88 [900/13861 (6%)]	Loss: 0.541779
Train Epoch: 88 [1200/13861 (9%)]	Loss: 0.539167
Train Epoch: 88 [1500/13861 (11%)]	Loss: 0.556653
Train Epoch: 88 [1800/13861 (13%)]	Loss: 0.524735
Train Epoch: 88 [2100/13861 (15%)]	Loss: 0.488812
Train Epoch: 88 [2400/13861 (17%)]	Loss: 0.511589
Train Epoch: 88 [2700/13861 (19%)]	Loss: 0.535711
Train Epoch: 88 [3000/13861 (22%)]	Loss: 0.529151
Train Epoch: 88 [3

Train Epoch: 91 [2700/13861 (19%)]	Loss: 0.547867
Train Epoch: 91 [3000/13861 (22%)]	Loss: 0.522721
Train Epoch: 91 [3300/13861 (24%)]	Loss: 0.535043
Train Epoch: 91 [3600/13861 (26%)]	Loss: 0.533426
Train Epoch: 91 [3900/13861 (28%)]	Loss: 0.528604
Train Epoch: 91 [4200/13861 (30%)]	Loss: 0.517301
Train Epoch: 91 [4500/13861 (32%)]	Loss: 0.518528
Train Epoch: 91 [4800/13861 (35%)]	Loss: 0.547610
Train Epoch: 91 [5100/13861 (37%)]	Loss: 0.520948
Train Epoch: 91 [5400/13861 (39%)]	Loss: 0.520835
Train Epoch: 91 [5700/13861 (41%)]	Loss: 0.516296
Train Epoch: 91 [6000/13861 (43%)]	Loss: 0.509190
Train Epoch: 91 [6300/13861 (45%)]	Loss: 0.532289
Train Epoch: 91 [6600/13861 (48%)]	Loss: 0.536672
Train Epoch: 91 [6900/13861 (50%)]	Loss: 0.530368
Train Epoch: 91 [7200/13861 (52%)]	Loss: 0.543153
Train Epoch: 91 [7500/13861 (54%)]	Loss: 0.537650
Train Epoch: 91 [7800/13861 (56%)]	Loss: 0.521385
Train Epoch: 91 [8100/13861 (58%)]	Loss: 0.524630
Train Epoch: 91 [8400/13861 (61%)]	Loss: 0.569317


Train Epoch: 94 [8100/13861 (58%)]	Loss: 0.539737
Train Epoch: 94 [8400/13861 (61%)]	Loss: 0.517739
Train Epoch: 94 [8700/13861 (63%)]	Loss: 0.538390
Train Epoch: 94 [9000/13861 (65%)]	Loss: 0.530140
Train Epoch: 94 [9300/13861 (67%)]	Loss: 0.537562
Train Epoch: 94 [9600/13861 (69%)]	Loss: 0.529507
Train Epoch: 94 [9900/13861 (71%)]	Loss: 0.534716
Train Epoch: 94 [10200/13861 (74%)]	Loss: 0.504867
Train Epoch: 94 [10500/13861 (76%)]	Loss: 0.523937
Train Epoch: 94 [10800/13861 (78%)]	Loss: 0.516938
Train Epoch: 94 [11100/13861 (80%)]	Loss: 0.498455
Train Epoch: 94 [11400/13861 (82%)]	Loss: 0.530963
Train Epoch: 94 [11700/13861 (84%)]	Loss: 0.525988
Train Epoch: 94 [12000/13861 (87%)]	Loss: 0.533526
Train Epoch: 94 [12300/13861 (89%)]	Loss: 0.509387
Train Epoch: 94 [12600/13861 (91%)]	Loss: 0.529946
Train Epoch: 94 [12900/13861 (93%)]	Loss: 0.554629
Train Epoch: 94 [13200/13861 (95%)]	Loss: 0.543956
Train Epoch: 94 [13500/13861 (97%)]	Loss: 0.524760
Train Epoch: 94 [13800/13861 (100%)]	L

Train Epoch: 97 [13500/13861 (97%)]	Loss: 0.523094
Train Epoch: 97 [13800/13861 (100%)]	Loss: 0.525542
====> Epoch: 97 Average train loss: 0.5260
====> Epoch: 97 Average test loss: 0.3638
Train Epoch: 98 [0/13861 (0%)]	Loss: 0.526311
Train Epoch: 98 [300/13861 (2%)]	Loss: 0.537913
Train Epoch: 98 [600/13861 (4%)]	Loss: 0.513260
Train Epoch: 98 [900/13861 (6%)]	Loss: 0.544094
Train Epoch: 98 [1200/13861 (9%)]	Loss: 0.503829
Train Epoch: 98 [1500/13861 (11%)]	Loss: 0.516575
Train Epoch: 98 [1800/13861 (13%)]	Loss: 0.550311
Train Epoch: 98 [2100/13861 (15%)]	Loss: 0.528225
Train Epoch: 98 [2400/13861 (17%)]	Loss: 0.553195
Train Epoch: 98 [2700/13861 (19%)]	Loss: 0.526909
Train Epoch: 98 [3000/13861 (22%)]	Loss: 0.512407
Train Epoch: 98 [3300/13861 (24%)]	Loss: 0.524494
Train Epoch: 98 [3600/13861 (26%)]	Loss: 0.506284
Train Epoch: 98 [3900/13861 (28%)]	Loss: 0.511150
Train Epoch: 98 [4200/13861 (30%)]	Loss: 0.526527
Train Epoch: 98 [4500/13861 (32%)]	Loss: 0.536567
Train Epoch: 98 [4800/1

In [39]:
# save model
torch.save(model, '../models/rnn_predict.pkl')

/anaconda3/envs/data_mining/lib/python3.6/site-packages/torch/serialization.py:241: UserWarning: Couldn't retrieve source code for container of type TS_rnn. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [41]:
train[:10]

(tensor([[[0., 0., 0.,  ..., 0., 1., 0.],
          [0., 0., 1.,  ..., 0., 0., 1.],
          [1., 0., 0.,  ..., 0., 1., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 1.],
          [1., 0., 0.,  ..., 0., 1., 0.],
          [1., 0., 0.,  ..., 0., 0., 1.]],
 
         [[0., 0., 0.,  ..., 0., 1., 0.],
          [0., 0., 0.,  ..., 0., 0., 1.],
          [0., 0., 1.,  ..., 0., 1., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 1.],
          [0., 0., 0.,  ..., 0., 0., 1.],
          [0., 0., 0.,  ..., 0., 1., 0.]],
 
         [[0., 1., 0.,  ..., 0., 0., 1.],
          [0., 1., 0.,  ..., 0., 0., 1.],
          [0., 0., 0.,  ..., 0., 0., 1.],
          ...,
          [0., 1., 0.,  ..., 0., 0., 1.],
          [0., 0., 0.,  ..., 0., 1., 0.],
          [0., 0., 1.,  ..., 0., 1., 0.]],
 
         ...,
 
         [[0., 0., 0.,  ..., 0., 1., 0.],
          [0., 1., 0.,  ..., 0., 0., 1.],
          [0., 0., 1.,  ..., 0., 0., 1.],
          ...,
          [0., 0., 0.,  ..., 0., 

In [61]:
a = torch.from_numpy(a)

In [62]:
b = torch.mean(a, 1)

In [66]:
b[:,0].unique()

tensor([376.0217, 353.1739, 378.8913, 394.0435, 485.5000, 509.7609, 588.5870,
        725.9565, 386.7609, 564.1304, 549.3913, 585.7826, 431.4348, 374.2826,
        696.2609, 342.8478, 440.7826, 488.7826, 418.8696, 397.4565, 357.8261,
        379.7609, 334.1522, 322.0217, 434.7391, 559.2609, 341.6522, 577.6957,
        419.1087, 565.3913])

In [68]:
c = np.load('../data/rnn_train_y.npy')
c = torch.from_numpy(c)

In [72]:
c

tensor([[[   1.,    1.,    4.],
         [   2.,  125.,    5.],
         [   1.,  813.,    4.],
         ...,
         [1519., 1365.,    2.],
         [ 743., 1039.,    4.],
         [1569., 1270.,    2.]],

        [[   1.,    1.,    2.],
         [   1.,  118.,    2.],
         [   1.,  235.,    4.],
         ...,
         [ 102., 1079.,    2.],
         [ 664., 1120.,    4.],
         [ 745., 1224.,    2.]],

        [[   1.,    1.,    4.],
         [   1.,  124.,    4.],
         [   1.,  143.,    2.],
         ...,
         [ 827.,  727.,    4.],
         [1361., 1477.,    4.],
         [ 827.,  852.,    4.]],

        ...,

        [[   1.,    1.,    4.],
         [   1.,   16.,    4.],
         [   1.,  119.,    2.],
         ...,
         [1285., 1063.,    4.],
         [1285., 1181.,    4.],
         [ 813.,    1.,    4.]],

        [[   1.,    5.,    2.],
         [   3.,   45.,    5.],
         [   1.,  130.,    4.],
         ...,
         [1141., 1340.,    4.],
         [11

In [205]:
pdist = torch.nn.PairwiseDistance(p=2)

In [206]:
a1 = torch.randn(4, 2)
a2 = torch.randn(4,2)

In [251]:
torch.mean(a1, 1)

tensor([-0.0431,  0.7919,  1.5557,  0.3310])

In [252]:
a1

tensor([[ 0.5227, -0.6088],
        [ 0.5556,  1.0283],
        [ 1.5431,  1.5683],
        [ 0.5531,  0.1089]])

In [210]:
a = pdist(a1,a2)

In [211]:
a

tensor([1.6103, 2.5548, 3.8726, 3.3924])

In [17]:
a = [1,2,3,4,5,6]

In [21]:
random.shuffle(a)

In [22]:
a

[6, 3, 2, 1, 5, 4]

In [34]:
df['Metric2'].value_counts()

1.0       176215
2.0        38878
3.0        21295
83.0       20960
43.0       13990
158.0      13009
84.0       10989
125.0      10989
167.0       9990
217.0       7992
663.0       7992
622.0       7992
408.0       6993
240.0       6016
137.0       5995
561.0       5994
443.0       5994
124.0       5994
57.0        5994
279.0       5994
647.0       5994
165.0       5975
160.0       5372
94.0        5028
774.0       5017
249.0       4995
745.0       4995
692.0       4995
138.0       4995
119.0       4983
           ...  
80.0           4
1666.0         4
923.0          4
1226.0         4
1678.0         4
1139.0         4
873.0          4
163.0          4
1051.0         4
726.0          4
90.0           2
427.0          1
154.0          1
415.0          1
251.0          1
210.0          1
278.0          1
1638.0         1
495.0          1
1595.0         1
1592.0         1
1575.0         1
799.0          1
967.0          1
825.0          1
845.0          1
879.0          1
333.0         